In [ ]:
%aiida
## GET THE TEXT
from xml.dom import minidom
import numpy as np
import copy
import re
import ipywidgets as ipw
import tempfile
from pybis import Openbis

# CLUSTER THE REACTIONS USING DBSCAN
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

## THINGS FOR RDIKIT (Not everything is needed)
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions #Only needed if modifying defaults
DrawingOptions.bondLineWidth=0.015
from rdkit.Chem import AllChem
from IPython.display import display, SVG
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D 
from openbabel import pybel as pybel

## THIS IS JUST TO OUTPUT ASCII TABLES
from prettytable import PrettyTable
from aiida_openbis.utils import bisutils,structures

from IPython.display import display, clear_output
from sklearn.decomposition import PCA
from ase import Atoms

from aiidalab_widgets_base import viewer
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, BasicStructureEditor
from apps.surfaces.widgets.import_cdxml import CdxmlUpload2GnrWidget

In [ ]:
bismols = structures.OpenbisMolWidget(title="BIS")
structure_selector = StructureManagerWidget(    
    importers=[bismols],
    #editors = [],
    storable=False, node_class='StructureData')
display(structure_selector)

In [ ]:
def make_ase(species, positions):
    """Create ase Atoms object."""
    # Get the principal axes and realign the molecule along z-axis.
    positions = PCA(n_components=3).fit_transform(positions)
    atoms = Atoms(species, positions=positions, pbc=True)
    atoms.cell = np.ptp(atoms.positions, axis=0) + 10
    atoms.center()

    return atoms

def _pybel_opt(smile, steps):
    """Optimize a molecule using force field and pybel (needed for complex SMILES)."""
    from openbabel import openbabel as ob
    from openbabel import pybel as pb

    obconversion = ob.OBConversion()
    obconversion.SetInFormat("smi")
    obmol = ob.OBMol()
    obconversion.ReadString(obmol, smile)

    pbmol = pb.Molecule(obmol)
    pbmol.make3D(forcefield="uff", steps=50)

    pbmol.localopt(forcefield="gaff", steps=200)
    pbmol.localopt(forcefield="mmff94", steps=100)

    f_f = pb._forcefields["uff"]  # pylint: disable=protected-access
    f_f.Setup(pbmol.OBMol)
    f_f.ConjugateGradients(steps, 1.0e-9)
    f_f.GetCoordinates(pbmol.OBMol)
    species = [chemical_symbols[atm.atomicnum] for atm in pbmol.atoms]
    positions = np.asarray([atm.coords for atm in pbmol.atoms])
    return make_ase(species, positions)

def _rdkit_opt(smile, steps):
    """Optimize a molecule using force field and rdkit (needed for complex SMILES)."""
    from rdkit import Chem
    from rdkit.Chem import AllChem

    smile = smile.replace("[", "").replace("]", "")
    mol = Chem.MolFromSmiles(smile)
    mol = Chem.AddHs(mol)

    AllChem.EmbedMolecule(mol, maxAttempts=20, randomSeed=42)
    AllChem.UFFOptimizeMolecule(mol, maxIters=steps)
    positions = mol.GetConformer().GetPositions()
    natoms = mol.GetNumAtoms()
    species = [mol.GetAtomWithIdx(j).GetSymbol() for j in range(natoms)]
    return make_ase(species, positions)

def mol_from_smiles(smile, steps=10000):
    """Convert SMILES to ase structure try rdkit then pybel"""
    try:
        return _rdkit_opt(smile, steps)
    except ValueError:
        return _pybel_opt(smile, steps)

In [ ]:
general_info  = ipw.HTML(
    value="<font size=3><b>General info</b></font>",
    #placeholder='Some HTML',
    #description='Some HTML',
)    
mol_number = ipw.IntText(
    value=325,
    description='Number:',
    disabled=False)

mol_batch = ipw.Dropdown(
    options=['a', 'b', 'c'],
    value='a',
    description='Batch:',
    disabled=False,
)

mol_acronym = ipw.Text(
    #value='Hello World',
    placeholder='DBBA',
    description='Acronym:',
    disabled=False
)

mol_iupac = ipw.Text(
    #value='Hello World',
    placeholder='10,10\'-dibromo..',
    description='IUPAC:',
    disabled=False
)


structure  = ipw.HTML(
    value="<font size=3><b>Structure</b></font>",
    #placeholder='Some HTML',
    #description='Some HTML',
)

mol_formula = ipw.Text(
    #value='Hello World',
    placeholder='C2H2',
    description='Formula:',
    disabled=False
)

mol_smiles = ipw.Text(
    #value='Hello World',
    placeholder='C1=CC=CC=C1',
    description='SMILES:',
    disabled=False
)

check_smiles = ipw.Button(
    description='Check',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

out_sketch = ipw.Output()

def generate_mol(c):
    with out_sketch:
        clear_output
        try:
            mol = Chem.MolFromSmiles(mol_smiles.value)
            AllChem.Compute2DCoords(mol)
            #show(mol,molSize=(375,75))
            molSize=(375,75)
            kekulize=True
            mc = Chem.Mol(mol.ToBinary())
            if kekulize:
                try:
                    Chem.Kekulize(mc)
                except:
                    mc = Chem.Mol(mol.ToBinary())
            assert mc.GetNumConformers() > 0
            drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
            drawer.DrawMolecule(mc)
            #drawer.bondLineWidth=0.005
            drawer.FinishDrawing()
            svg = drawer.GetDrawingText()
            display(SVG(svg.replace('svg:','')))
            mol_formula.value=mol_from_smiles('C=C').get_chemical_formula()
        except:
            mol_formula.value=''
    
check_smiles.on_click(generate_mol)

mol_cas = ipw.Text(
    #value='Hello World',
    placeholder='121848-75-7',
    description='CAS Number:',
    disabled=False
)



properties  = ipw.HTML(
    value="<font size=3><b>Properties</b></font>",
    #placeholder='Some HTML',
    #description='Some HTML',
)

mol_evaporation = ipw.IntText(
    value=220,
    description='Evaporation T (C):',
    disabled=False)

mol_hazardous = ipw.Checkbox(value=False, description='Hazardous',
                                 tooltip='Activate HAZARD', 
                                 style={'description_width': '80px'}) 

mol_hazard = ipw.Text(
    #value='Hello World',
    placeholder='volatile',
    description='Hazard type:',
    disabled=False
)

out_hazard = ipw.Output()

safety_upload = ipw.FileUpload(description='Upload safety data sheet(s)', multiple=True, layout={'width': 'initial'})
        
def on_hazard(c=None):
    with out_hazard:
        clear_output()
        if mol_hazardous.value:
            display(ipw.VBox([mol_hazard,safety_upload]))
        else:
            mol_hazard.value=''

mol_hazardous.observe(on_hazard, 'value')        



storage  = ipw.HTML(
    value="<font size=3><b>Storage conditions</b></font>",
    #placeholder='Some HTML',
    #description='Some HTML',
)


mol_fridge = ipw.Checkbox(
    value=False,
    description='Fridge',
    disabled=False,
    indent=False
)


mol_nolight = ipw.Checkbox(
    value=False,
    description='No light',
    disabled=False,
    indent=False
)

mol_dry = ipw.Checkbox(
    value=False,
    description='Dry',
    disabled=False,
    indent=False
)

mol_noo2 = ipw.Checkbox(
    value=False,
    description='No O2',
    disabled=False,
    indent=False
)

out_storeother = ipw.Output()

mol_storeother = ipw.Checkbox(
    value=False,
    description='Other',
    disabled=False,
    indent=False
)

mol_storeotherdescription = ipw.Text(
    #value='Hello World',
    placeholder='special fridge',
    description='Storage type:',
    disabled=False
)

def on_storeother(c=None):
    with out_storeother:
        clear_output()
        if mol_storeother.value:
            display(mol_storeotherdescription)
        else:
            mol_storeotherdescription.value=''    

mol_storeother.observe(on_storeother,'value')

supplier  = ipw.HTML(
    value="<font size=3><b>Supplier info</b></font>",
    #placeholder='Some HTML',
    #description='Some HTML',
)

mol_supplier = ipw.Text(
    #value='Hello World',
    placeholder='Feng Group (Dresden)',
    description='Supplier:',
    disabled=False
)

mol_synthesisedby = ipw.Text(
    #value='Hello World',
    placeholder='Qiang Huang',
    description='Synth. by:',
    disabled=False
)

mol_supownname = ipw.Text(
    #value='Hello World',
    placeholder='QC76',
    description='Supplier own name:',
    disabled=False
)

mol_amount = ipw.IntText(
    value=100,
    description='amount (mg):',
    disabled=False)

mol_receivedate = ipw.DatePicker(
    description='Receive date:',
    disabled=False
)

additional_info  = ipw.HTML(
    value="<font size=3><b>Additional info</b></font>",
    #placeholder='Some HTML',
    #description='Some HTML',
)

mol_addinfo = ipw.Textarea(
    value='',
    placeholder='Type something',
    description='Addiitonal Info:',
    disabled=False
)

mol_store = ipw.Button(
    description='Store',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

store_msg = ipw.HTML(value='')

#def on_safety_upload(change=None):
#    tmpdir = tempfile.mkdtemp()
#    file_path = tmpdir + "/" + 'reaction.cdxml'
#    with open(file_path, 'w+b') as f:
#        f.write(aa.data[0])
#    lmolecules,lreactions = parsecdxml(file_path)
#    getsmilesfromcdxml(file_path, lmolecules)
#    bisutils.new_reaction_products(reactions=lreactions, molecules=lmolecules, attachment=file_path)
#
#
def on_mol_store(c=None):
    date=str(mol_receivedate.value.year)+'-'+str(mol_receivedate.value.month)+'-'+str(mol_receivedate.value.day)
    
    all_files=[]
    if True : #not str(mol_number.value)+str(mol_batch.value) in bismols.smiles._options_labels:
        if len(safety_upload.data)>0 or len(photo_upload.data) >0:
            tmpdir = tempfile.mkdtemp()
            for metadata, file in zip(safety_upload.metadata,safety_upload.data):
                file_path = tmpdir + "/" + 'safety_'+metadata['name']
                all_files.append(file_path)
                with open(file_path, 'w+b') as f:
                    f.write(file)
            for metadata, file in zip(photo_upload.metadata,photo_upload.data):
                file_path = tmpdir + "/" + 'photo_'+metadata['name']
                all_files.append(file_path)
                with open(file_path, 'w+b') as f:
                    f.write(file)


        bisutils.new_molecule_precursor(session=None, 
                                        number=mol_number.value, 
                                        batch=mol_batch.value,
                                        acronym=mol_acronym.value,
                                        iupac=mol_iupac.value,
                                        chemformula=mol_formula.value,
                                        smile=mol_smiles.value,
                                        cas=mol_cas.value,
                                        evaporationt=mol_evaporation.value,
                                        hazardous=mol_hazardous.value,
                                        azardousdescription=mol_hazard.value,
                                        fridge=mol_fridge.value,
                                        nolight=mol_nolight.value,
                                        dry=mol_dry.value,
                                        nooxygen=mol_noo2.value,
                                        otherstorage=mol_storeother.value,
                                        specifyotherstorage=mol_storeotherdescription.value,
                                        supplier=mol_supplier.value,
                                        synthesizedby=mol_synthesisedby.value,
                                        supplierownname=mol_supownname.value,
                                        amount=mol_amount.value,
                                        receivingdate=date,
                                        addcomments=mol_addinfo.value,
                                        filestoupload=all_files
                                        )
    else:
        store_msg.value = "<font size=3><b>Molecule already in database</b></font>"
    
mol_store.on_click(on_mol_store)

photo_upload = ipw.FileUpload(description='Upload photo', multiple=True, layout={'width': 'initial'})
#photo_upload.observe(on_file_upload, names='value')
info = ipw.VBox([general_info,ipw.HBox([mol_number,mol_batch]),mol_acronym,mol_iupac,
                 structure, mol_formula,ipw.HBox([mol_smiles,check_smiles]),mol_cas,out_sketch,
                 properties,mol_hazardous,out_hazard,mol_evaporation,
                 storage,ipw.HBox([mol_fridge,mol_nolight,mol_dry,mol_noo2,mol_storeother]),out_storeother,
                 supplier,mol_supplier,mol_synthesisedby,mol_supownname,mol_amount,mol_receivedate,
                 additional_info,mol_addinfo,
                 photo_upload,
                 mol_store,store_msg
                ])
display(info)


In [ ]:
from aiida_openbis.utils import bisutils

if False:
    session = bisutils.log_in()
    for collection in ['/MATERIALS/MOLECULES/PRECURSORS',
                       #'/MATERIALS/SAMPLES/MOLECULES',
                       #'/MATERIALS/SAMPLES/CHEMSKETCH'
                      ]:
        objects = session.get_collection(collection).get_objects()
        for obj in objects:
            print(obj.permId)
            obj.delete('cleanup')       
    session.logout()

In [ ]:
#session = bisutils.log_in()

In [ ]:
#allm = bisutils.get_precursors(session=session)

In [ ]:
#mo = session.get_object('20211022095519907-831')

In [ ]:
#mo.props['molecule.receivingdate']

In [ ]:
#rawds = session.new_dataset(type='RAW_DATA', object=mo, file='/tmp/tmpo2_0oge_/photo_20210817-openBIS-admin-training.pdf')
#rawds.save()

In [ ]:
#session.logout()